In [1]:
import intake
import pandas as pd

df = intake.open_csv('./data/bird_migration/{species}.csv').read()

def fill_day(v):
    next_year = v.assign(day=v.day + v.day.max())
    last_year = v.assign(day=v.day - v.day.max())
    surrounding_years = pd.concat([last_year, v, next_year])
    filled = surrounding_years.assign(
        lat=surrounding_years.lat.interpolate(), 
        lon=surrounding_years.lon.interpolate())
    this_year = filled[filled.day.isin(v.day)]
    return this_year

df = pd.concat([fill_day(v) for k, v in df.groupby('species')])

## Geoviews

In [8]:
import holoviews as hv
import geoviews as gv

import geoviews.tile_sources as gts
import cartopy.crs as ccrs
hv.extension('bokeh')

In [9]:
bird_ds = gv.Dataset(df, kdims=['lon', 'lat'], vdims=['day', 'species'], crs=ccrs.PlateCarree())

In [15]:
%%opts Points [color_index='day' height=500 width=400 show_legend=False] (size=1)

p = bird_ds.to(gv.Points)
p * gv.feature.coastline

:Overlay
   .Points.I    :Points   [lon,lat]   (day,species)
   .Coastline.I :Feature   [Longitude,Latitude]

**NOTE:** that the y-axis has changed

In [16]:
print(p)

:Points   [lon,lat]   (day,species)


We can also overlay tiles:

In [21]:
grouped_birds = p.groupby('day', dynamic=True)
print(grouped_birds)

:DynamicMap   [day]


In [22]:
%%opts Points [color_index='species' height=500 width=400 show_legend=False, tools=['hover']] (cmap='rainbow' size=5)

grouped_birds * gv.feature.coastline

:DynamicMap   [day]
   :Overlay
      .Points.I    :Points   [lon,lat]   (day,species)
      .Coastline.I :Feature   [Longitude,Latitude]

Hmmm but that is out of bounds. Let's put that over tiles

In [23]:
tiles = gts.EsriImagery()
tiles.extents = df.lon.min(), df.lat.min(), df.lon.max(), df.lat.max()

In [95]:
styled_birds = grouped_birds.options(color_index='species', height=500, width=400, 
                                     show_legend=False, tools=['hover'], 
                                     cmap='rainbow', size=5)

In [96]:
tiles * styled_birds

:DynamicMap   [day]
   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [lon,lat]   (day,species)

### Equivalent in hvplot

In [ ]:
import hvplot.pandas

In [204]:
%%output holomap="scrubber"

filled_days.hvplot.points(x='lon', y='lat', groupby='day', color='species', 
                          cmap='rainbow', legend=False, geo=True,
                          height=500, width=400) * tiles

:DynamicMap   [day]
   :Overlay
      .Points.I :Points   [lon,lat]   (species)
      .WMTS.I   :WMTS   [Longitude,Latitude]

## Adding another layer

Now let's put it over our air temperature data. We'll set it up the same way as before, but we'll include `geo=True` in the arguments.

In [78]:
import xarray as xr
import hvplot.xarray

ds = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep/air.day.ltm.nc')
ds = ds.rename(time='day').sel(level=1000)
ds['day'] = list(range(1,366))

/Users/jsignell/xarray/xarray/coding/times.py:122: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/Users/jsignell/xarray/xarray/coding/variables.py:69: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  return self.func(self.array)


In [100]:
%output holomap="scrubber"

grouped_air = ds.hvplot('lon', 'lat', groupby='day', geo=True, height=600)
grouped_air * styled_birds * gv.feature.coastline

:DynamicMap   [day]
   :Overlay
      .Image.I     :Image   [lon,lat]   (air)
      .Points.I    :Points   [lon,lat]   (day,species)
      .Coastline.I :Feature   [Longitude,Latitude]

[Next Section](./04_panel.ipynb#panel)